# Scrapper para obtener cédula profesional

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
import pandas as pd
from utils.functions import *
import requests
from unidecode import unidecode

## Conexión al navegador

In [3]:
chrome_options = Options()
driver = webdriver.Chrome(options=chrome_options)
start_url = 'https://cedulaprofesionalsep.online/#Consulta_de_Cedula_Profesional'
driver.get(start_url)
# Modificar el tamaño de la ventana
#driver.execute_script("document.body.style.zoom='50%'")

Error getting version of chromedriver 117. Retrying with chromedriver 116 (attempt 1/5)
Error getting version of chromedriver 116. Retrying with chromedriver 115 (attempt 2/5)
Error getting version of chromedriver 115. Retrying with chromedriver 114 (attempt 3/5)


SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 114
Current browser version is 117.0.5938.134 with binary path C:\Program Files\Google\Chrome\Application\chrome.exe
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0150A813+48355]
	(No symbol) [0x0149C4B1]
	(No symbol) [0x013A5358]
	(No symbol) [0x013C61AC]
	(No symbol) [0x013C1EF3]
	(No symbol) [0x013C0579]
	(No symbol) [0x013F0C55]
	(No symbol) [0x013F093C]
	(No symbol) [0x013EA536]
	(No symbol) [0x013C82DC]
	(No symbol) [0x013C93DD]
	GetHandleVerifier [0x0176AABD+2539405]
	GetHandleVerifier [0x017AA78F+2800735]
	GetHandleVerifier [0x017A456C+2775612]
	GetHandleVerifier [0x015951E0+616112]
	(No symbol) [0x014A5F8C]
	(No symbol) [0x014A2328]
	(No symbol) [0x014A240B]
	(No symbol) [0x01494FF7]
	BaseThreadInitThunk [0x75F105C9+25]
	RtlGetAppContainerNamedObjectPath [0x77C277AD+237]
	RtlGetAppContainerNamedObjectPath [0x77C2777D+189]


> Función para obtener el tamaño de una solicitud HTTP

In [30]:
def get_request_size(url):
    response = requests.get(url)
    return len(response.content)

## Leer archivo CSV y obtener lista de nombres

In [7]:
init=2
final=500

df = pd.read_excel('Z:/Data/HSBC/Asignaciones/ASG_TDC_IA_ENERO_JUNIO.xlsx')
df['Nombre'] = df.Nombre.apply(lambda X: X.replace('/', ' ').replace('*', '').rstrip())
df_filtered = df['Nombre']
df_filtered.drop_duplicates(inplace=True)
df_input = pd.DataFrame(df_filtered.reset_index(drop=True)).iloc[init:final]



df_input['NombreSplit'] = df_input.Nombre.apply(lambda X: split_name(X))
df_input['Nombre'] = df_input.NombreSplit.apply(lambda X: X[0])
df_input['Paterno'] = df_input.NombreSplit.apply(lambda X: X[1])
df_input['Materno'] = df_input.NombreSplit.apply(lambda X: X[2])

# Concatenación
df_input['Nombre_Completo'] = df_input.apply(lambda row: ' '.join([row['Nombre'], row['Paterno'], row['Materno']]), axis = 1)
# Cambiar nombre
#df.rename(columns={'Nombre':'Nombre'}, inplace=True)
df_input.tail()

TypeError: sequence item 2: expected str instance, NoneType found

In [4]:
df_input.head()

,Nombre,NombreSplit,Paterno,Materno,Nombre_Completo
192,jose de jesus,"[jose de jesus, hernandez, hernandez]",hernandez,hernandez,jose de jesus hernandez hernandez
193,jose adan,"[jose adan, perez, tamayo]",perez,tamayo,jose adan perez tamayo
194,alejandro,"[alejandro, perez, alcantara]",perez,alcantara,alejandro perez alcantara
195,j ascencion,"[j ascencion, pina, patino]",pina,patino,j ascencion pina patino
196,miguel angel,"[miguel angel, sanchez, ramos]",sanchez,ramos,miguel angel sanchez ramos


### Scrapper

In [5]:
# Código para modificar la ventana
driver.execute_script("window.scrollTo(10, 7600);")

In [13]:
data=[]
#total_data_downloaded = 0
for index, row in df_input.iterrows():
    nombre_persona = row['Nombre_Completo']
   
    # Se escrolea hasta la parte específica 
    driver.execute_script("window.scrollTo(10, 7600);")
    time.sleep(5)
    
    # Se ingresan las variables del scrapeo
    nombre=row['Nombre']
    paterno=row['Paterno']
    materno=row['Materno']
    
    # Ingresar nombres
    celda_nombre = driver.find_element(By.XPATH, '//*[@id="input-nombre"]')
    celda_nombre.clear()
    celda_nombre.send_keys(str(nombre))

    #Ingresar Apellido Paterno
    celda_paterno = driver.find_element(By.XPATH, '//*[@id="input-apaterno"]')
    celda_paterno.clear()
    celda_paterno.send_keys(str(paterno))

    # Ingresar Apellido Materno
    celda_materno = driver.find_element(By.XPATH, '//*[@id="input-amaterno"]')
    celda_materno.clear()
    celda_materno.send_keys(str(materno))

    # Click en botón buscar
    buscar = driver.find_element(By.XPATH, '//*[@id="container-form-1"]/form/div[4]/div/input')
    buscar.click()
    time.sleep(2)

    max_wait_time = 5  # Tiempo máximo de espera en segundos
    start_time = time.time()  # Momento de inicio de espera

    while time.time() - start_time < max_wait_time:
        try:
            alert = driver.switch_to.alert
            alert.accept()
            driver.refresh()
            break  # Salir del bucle si se encuentra la ventana emergente
        except:

            time.sleep(1)  # Esperar 1 segundo antes de intentar nuevamente
            
            


    time.sleep(5)
    
    # Esperar a que los elementos estén visibles
    ver_botones = WebDriverWait(driver, 10).until(EC.visibility_of_all_elements_located((By.CLASS_NAME, 'ver-mas')))

    
    for ver in ver_botones:
        nombre_boton = ver.get_attribute("value")
        print(nombre_boton)
        if unidecode(nombre_boton.lower()) == unidecode(nombre_persona.lower()):
            cantidad_registros = len(ver_botones)

            ver.click()
            
            # Encontrar los elementos de entrada de texto por su id
            input_cedula = driver.find_element(By.ID, "input-cedula-result")
            input_tipo = driver.find_element(By.ID, "input-tipo-result")
            input_sexo = driver.find_element(By.ID, "input-sexo-result")
            input_nombre = driver.find_element(By.ID, "input-nombre-result")
            input_paterno = driver.find_element(By.ID, "input-paterno-result")
            input_materno = driver.find_element(By.ID, "input-materno-result")
            input_escuela = driver.find_element(By.ID, "input-escuela-result")
            input_titulo = driver.find_element(By.ID, "input-titulo-result")

            # Obtener los valores de los campos
            valor_cedula = input_cedula.get_attribute("value")
            valor_tipo = input_tipo.get_attribute("value")
            valor_sexo = input_sexo.get_attribute("value")
            valor_nombre = input_nombre.get_attribute("value")
            valor_paterno = input_paterno.get_attribute("value")
            valor_materno = input_materno.get_attribute("value")
            valor_escuela = input_escuela.get_attribute("value")
            valor_titulo = input_titulo.get_attribute("value")

            # Agregar los valores a la lista de diccionarios
            data.append({
            "Cédula": valor_cedula,
            "Tipo de Cédula": valor_tipo,
            "Sexo": valor_sexo,
            "Nombre": valor_nombre,
            "Paterno": valor_paterno,
            "Materno": valor_materno,
            "Escuela": valor_escuela,
            "Título": valor_titulo
            })
            # Encontrar los elementos de entrada de texto por su id
            cerrar = driver.find_element(By.XPATH, '//*[@id="cerrar_resut_personal"]')
            cerrar.click()
       
        
        # Obtener el tamaño de la página descargada
        #page_source_size = len(driver.page_source.encode('utf-8'))
        #total_data_downloaded += page_source_size
        
driver.quit()            
df_data = pd.DataFrame(data)
df_data.to_csv('Z:/Data/HSBC/SCRAPPING/Cédulas/0_500_.csv')
    

                



TimeoutException: Message: 


In [52]:
df_data.tail(10)

,Cédula,Tipo de Cédula,Sexo,Nombre,Paterno,Materno,Escuela,Título
953,12817006,C1,Femenino,MARIA DE JESUS,HERNANDEZ,HERNANDEZ,UNIVERSIDAD AUTÓNOMA DEL ESTADO DE MÉXICO,LICENCIATURA EN ENFERMERÍA
954,13305539,C1,Maculino,EDGAR DE JESUS,HERNANDEZ,HERNANDEZ,INSTITUTO TECNOLÓGICO DE CIUDAD VALLES,LICENCIATURA EN INGENIERÍA INDUSTRIAL
955,0012044,A1,Maculino,J. JESÚS,HERNÁNDEZ,HERNÁNDEZ,UNIVERSIDAD NACIONAL AUTÓNOMA DE MÉXICO,ESPECIALIDAD EN GINECOLOGÍA Y OBSTETRICIA
956,8318023,C1,Femenino,TERESA DE JESUS,HERNANDEZ,HERNANDEZ,UNIVERSIDAD MERIDIONAL (UNIVERSIDAD MAYA),LICENCIATURA EN PEDAGOGÍA
957,0515938,C1,Maculino,JESÚS ALBERTO,HERNÁNDEZ,HERNÁNDEZ,UNIVERSIDAD AUTÓNOMA DE GUADALAJARA,LICENCIATURA EN ARQUITECTURA
958,0337962,C1,Maculino,JESÚS,HERNÁNDEZ,HERNÁNDEZ,BENEMÉRITA UNIVERSIDAD AUTÓNOMA DE PUEBLA,LICENCIATURA COMO MÉDICO CIRUJANO Y PARTERO
959,0103810,C1,Maculino,JESÚS,HERNÁNDEZ,HERNÁNDEZ,UNIVERSIDAD NACIONAL AUTÓNOMA DE MÉXICO,LICENCIATURA EN DERECHO
960,8257868,C1,Maculino,JESÚS MANUEL,HERNÁNDEZ,HERNÁNDEZ,UNIVERSIDAD TECNOLÓGICA DE QUERÉTARO,TEC. SUP. UNIVER. EN MANTENIMIENTO INDUSTRIAL
961,0491940,C1,Maculino,FRANCISCO DE JESÚS,HERNÁNDEZ,HERNÁNDEZ,UNIVERSIDAD LA SALLE,LICENCIATURA COMO CONTADOR PÚBLICO
962,8401465,C1,Maculino,JESUS,HERNANDEZ,HERNANDEZ,INSTITUTO TECNOLOGICO SUPERIOR DE TAMAZUNCHALE,LICENCIATURA EN INGENIERÍA INDUSTRIAL


In [37]:
nombre_buscado = "jose de jesus"
resultado = df_input.loc[df_input['Nombre'] == nombre_buscado]
resultado

,Nombre,NombreSplit,Paterno,Materno
192,jose de jesus,"[jose de jesus, hernandez, hernandez]",hernandez,hernandez


In [39]:
# Convertir los datos en un DataFrame y guardar en CSV              
df_data = pd.DataFrame(data)
df_data.to_csv('Z:/Data/HSBC/SCRAPPING/Cédulas/0_192_.csv')
    

# Scraper marcatel

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os
from datetime import datetime, timedelta, date
from dotenv import load_dotenv, find_dotenv
import pandas as pd
import json
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import warnings
warnings.filterwarnings('ignore')

def marcatel_automation():
    url = "https://tink.marcatel.com.mx/Login.aspx"
    
    load_dotenv(find_dotenv()) # Load the .env file.
    userid = os.getenv("USER_MARCATEL")
    passwo = os.getenv("PASSWORD_MARCATEL")
    
   
    chrome_options = Options()
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)
    wait = WebDriverWait(driver, 10)
    user = wait.until(EC.presence_of_element_located((By.ID, "usuario")))
    user.click()
    user.send_keys(userid)
    contra = wait.until(EC.presence_of_element_located((By.ID, "contrasena")))
    
        # Pasar credenciales
    contra.click()
    contra.send_keys(passwo)
    init_button = wait.until(EC.element_to_be_clickable((By.ID, "btnini")))
    init_button.click()
    time.sleep(10)
        # Abrir ventana de reportes SMS
        # Redirecciona a la URL deseada
    driver.get("https://tink.marcatel.com.mx/Reportes/wfReporteRespuestas.aspx")
        # Espera unos segundos antes de cerrar el navegador
    time.sleep(10)
    
    # Get current date and time
    current_date = datetime.now()
    
    # Yesterday's date
    yersterday_date = current_date - timedelta(days=1)
    
    # Format date
    formatted_date = yersterday_date.strftime("%Y-%m-%d")
    
    # Date to marcatel
   
    
    time.sleep(6)

    reporte = wait.until(EC.element_to_be_clickable((By.ID, "generarReporte")))
    reporte.click()
    time.sleep(30)
    exportar = driver.find_element(By.ID, "excel")
    driver.execute_script("arguments[0].scrollIntoView();", exportar)
    exportar.click()
    time.sleep(10)
    driver.quit()
marcatel_automation()
